In [ ]:
# default_exp models.indrnn
# default_cls_lvl 3

# IndRNN
> Pytorch Models for Sequential Data

In [ ]:
#export
from seqdata.core import *
from fastai2.basics import *

import sys
import time
import math
#import warnings

import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Function

from torch.nn import Parameter
from cupy.cuda import function
from pynvrtc.compiler import Program
from collections import namedtuple

In [ ]:
seq = DataBlock(blocks=(SequenceBlock.from_hdf(['current','voltage'],TensorSequencesInput,clm_shift=[0,-1]),
                        SequenceBlock.from_hdf(['voltage'],TensorSequencesOutput,clm_shift=[-1])),
                 get_items=CreateDict([DfHDFCreateWindows(win_sz=1000+1,stp_sz=50,clm='current')]),
                 splitter=ApplyToDict(ParentSplitter()))
db = seq.dataloaders(get_hdf_files('/mnt/data/Development/seqdata/test_data/'))

## IndRNN Cuda Modules

In [ ]:
#export

IndRNN_CODE = """
extern "C" {

    __forceinline__ __device__ float reluf(float x)
    {
        return (x > 0.f) ? x : 0.f;
    }

    __forceinline__ __device__ float calc_grad_activation(float x)
    {
        return (x > 0.f) ? 1.f : 0.f;
    }

    __global__ void indrnn_fwd( const float * __restrict__ x,
                            const float * __restrict__ weight_hh, const float * __restrict__ h0,
                            const int len, const int batch, const int hidden_size, 
                            float * __restrict__ h)
    {
        int ncols = batch*hidden_size;
        int col = blockIdx.x * blockDim.x + threadIdx.x;
        if (col >= ncols) return;       
        const float weight_hh_cur = *(weight_hh + (col%hidden_size));
        float cur = *(h0 + col);
        const float *xp = x+col;
        float *hp = h+col;

        for (int row = 0; row < len; ++row)
        {
            cur=reluf(cur*weight_hh_cur+(*xp));
            *hp=cur;
            xp += ncols;
            hp += ncols;            
        }
    }

    __global__ void indrnn_bwd(const float * __restrict__ x,
                             const float * __restrict__ weight_hh, const float * __restrict__ h0,
                             const float * __restrict__ h,
                            const float * __restrict__ grad_h, 
                            const int len, const int batch, const int hidden_size, 
                            float * __restrict__ grad_x,
                            float * __restrict__ grad_weight_hh, float * __restrict__ grad_h0)
    {    
        int ncols = batch*hidden_size;
        int col = blockIdx.x * blockDim.x + threadIdx.x;
        if (col >= ncols) return;        
        const float weight_hh_cur = *(weight_hh + (col%hidden_size));
        float gweight_hh_cur = 0;
        float cur = 0;  // *(grad_last + col);        //0; strange gradient behavior. grad_last and grad_h, one of them is zero.     
        
        const float *xp = x+col + (len-1)*ncols;
        const float *hp = h+col + (len-1)*ncols;      
        float *gxp = grad_x + col + (len-1)*ncols;
        const float *ghp = grad_h + col + (len-1)*ncols;
        

        for (int row = len-1; row >= 0; --row)
        {        
            const float prev_h_val = (row>0) ? (*(hp-ncols)) : (*(h0+col));
            //float h_val_beforeact = prev_h_val*weight_hh_cur+(*xp);
            float gh_beforeact = ((*ghp) + cur)*calc_grad_activation(prev_h_val*weight_hh_cur+(*xp));
            cur = gh_beforeact*weight_hh_cur;
            gweight_hh_cur += gh_beforeact*prev_h_val;
            *gxp = gh_beforeact;

            xp -= ncols;
            hp -= ncols;
            gxp -= ncols;
            ghp -= ncols;        
        }

        atomicAdd(grad_weight_hh + (col%hidden_size), gweight_hh_cur);
        *(grad_h0 +col) = cur;
    }
}
"""

In [ ]:
#export
def _get_indrnn_cuda(device):
    if not hasattr(_get_indrnn_cuda, '_DEVICE2FUNC'):_get_indrnn_cuda._DEVICE2FUNC = {}
    res = _get_indrnn_cuda._DEVICE2FUNC.get(device, None)
    if res:
        return res
    else:
        print ('IndRNN loaded for gpu {}'.format(device))
        mod = function.Module()
        mod.load(bytes(Program(IndRNN_CODE, 'indrnn_prog.cu').compile().encode()))
        fwd_func = mod.get_function('indrnn_fwd')
        bwd_func = mod.get_function('indrnn_bwd')

        Stream = namedtuple('Stream', ['ptr'])
        current_stream = Stream(ptr=torch.cuda.current_stream().cuda_stream)

        _get_indrnn_cuda._DEVICE2FUNC[device] = (current_stream, fwd_func, bwd_func)
        return current_stream, fwd_func, bwd_func

class IndRNN_Compute_GPU(Function):

    @staticmethod
    def forward(ctx, x, weight_hh, h0):
        length = x.size(0) if x.dim() == 3 else 1
        batch = x.size(-2)
        hidden_size = x.size(-1)  #hidden_size
        ncols = batch*hidden_size
        thread_per_block = min(512, ncols)
        num_block = (ncols-1)//thread_per_block+1
        
        size = (length, batch, hidden_size) if x.dim() == 3 else (batch, hidden_size)
        h = x.new(*size)

        stream, fwd_func, _ = _get_indrnn_cuda(x.device)
        FUNC = fwd_func
        FUNC(args=[
            x.contiguous().data_ptr(),
            weight_hh.contiguous().data_ptr(),
            h0.contiguous().data_ptr(),
            length,
            batch,
            hidden_size,
            h.contiguous().data_ptr()],
            block = (thread_per_block,1,1), grid = (num_block,1,1),
            stream=stream
        )

        ctx.save_for_backward(x, h, weight_hh, h0)#
        return h

    @staticmethod
    def backward(ctx, grad_h):
        x, h, weight_hh, h0 = ctx.saved_tensors
        length = x.size(0) if x.dim() == 3 else 1
        batch = x.size(-2)
        hidden_size = x.size(-1)#self.hidden_size
        ncols = batch*hidden_size
        thread_per_block = min(512, ncols)
        num_block = (ncols-1)//thread_per_block+1

        grad_x = x.new(*x.size())
        grad_weight_hh = x.new(hidden_size).zero_()
        grad_h0 = x.new(batch, hidden_size)  

        stream, _, bwd_func = _get_indrnn_cuda(x.device)
        FUNC = bwd_func
        FUNC(args=[
            x.contiguous().data_ptr(),
            weight_hh.contiguous().data_ptr(),
            h0.contiguous().data_ptr(),
            h.contiguous().data_ptr(),
            grad_h.contiguous().data_ptr(),
            length,
            batch,
            hidden_size,
            grad_x.contiguous().data_ptr(),
            grad_weight_hh.contiguous().data_ptr(),
            grad_h0.contiguous().data_ptr()],
            block = (thread_per_block,1,1), grid = (num_block,1,1),
            stream=stream
        )
        return grad_x, grad_weight_hh, grad_h0

In [ ]:
#export
class IndRNN_onlyrecurrent(nn.Module):
    def __init__(self, hidden_size, recurrent_init=None):
        super(IndRNN_onlyrecurrent, self).__init__()
        self.hidden_size = hidden_size
        self.recurrent_init = recurrent_init
        self.weight_hh = Parameter(torch.Tensor(hidden_size))        
        self.reset_parameters()

    def reset_parameters(self):
        for name, weight in self.named_parameters():
            if "weight_hh" in name: nn.init.uniform_(weight, a=0, b=1)

    def forward(self, input, h0=None):
        assert input.dim() == 2 or input.dim() == 3        
        if h0 is None:
            h0 = input.data.new(input.size(-2),input.size(-1)).zero_()
        elif (h0.size(-1)!=input.size(-1)) or (h0.size(-2)!=input.size(-2)):
            raise RuntimeError(
                'The initial hidden size must be equal to input_size. Expected {}, got {}'.format(
                    h0.size(), input.size()))
        IndRNN_Compute = IndRNN_Compute_GPU.apply
        #h=IndRNN_Compute(input, self.weight_hh, h0)
        return IndRNN_Compute(input, self.weight_hh, h0)

In [ ]:
#export
class Linear_overtime_module(nn.Module):
    def __init__(self, input_size, hidden_size,bias=True):
        super().__init__()
        self.fc = nn.Linear(input_size, hidden_size, bias=bias)
        self.input_size = input_size
        self.hidden_size = hidden_size

    def forward(self, x):
        y = x.contiguous().view(-1, self.input_size)
        y = self.fc(y)
        y = y.view(x.size()[0], x.size()[1], self.hidden_size)
        return y

In [ ]:
#export
# from seqdata.models.core import SeqLinear
class IndRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1,batch_first=True):
        super().__init__()        
        self.batch_first = batch_first
        
#         self.list_linear=nn.ModuleList([SeqLinear(input_size if i == 0 else hidden_size, hidden_size,hidden_layer=0) for i in range(num_layers)])
        self.list_linear=nn.ModuleList([Linear_overtime_module(input_size if i == 0 else hidden_size, hidden_size) for i in range(num_layers)])
        self.list_rnn = nn.ModuleList([IndRNN_onlyrecurrent(hidden_size) for i in range(num_layers)])     
            
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'weight_hh' in name:
                param.data.uniform_(0,0.99)            
            if ('list_linear' in name) and 'weight' in name:#'denselayer' in name and 
                nn.init.kaiming_uniform_(param, a=8, mode='fan_in')#
            if 'bias' in name:
                param.data.fill_(0.0)
    
    def forward(self, inp, hid=None):
#         import pdb;pdb.set_trace()
        new_hid = []
        if self.batch_first: inp = inp.transpose(0,1)
        for i, (lin,rnn) in enumerate(zip(self.list_linear,self.list_rnn)):
            inp = lin(inp)
            inp = rnn(inp , None if hid is None else hid[i])
            new_hid.append(inp[-1])
            
        if self.batch_first: inp = inp.transpose(0,1)
        return inp, torch.stack(new_hid, 0)
      

In [ ]:
#export
class IndRNNRegularizer(Callback):
    "Callback that adds ||max(0,|U|-U_bound)|| to the loss, only useful for IndRNN"
    run_before=TrainEvalCallback
    def __init__(self, clip_limit = 0.99):
        super().__init__()
        self.clip_limit = clip_limit
        self.zero = None
        
    def after_loss(self):
#         ||max(0,|U|-U_bound)||
        if not self.training: return
        if self.zero is None: self.zero = list(self.model.parameters())[0].new_tensor(0.)
        
        for m in self.model.modules():
            if type(m) == IndRNN_onlyrecurrent:
#                 import pdb;pdb.set_trace()
                self.learn.loss += torch.max(self.zero,m.weight_hh.abs()-self.clip_limit).norm()

In [ ]:
class SimpleIndRNN(nn.Module):
    
    def __init__(self,input_size,output_size,num_layers=1,hidden_size=100,bn=False,seq_len=1000):
        super().__init__()
        self.rnn = IndRNN(input_size=input_size,hidden_size=hidden_size,num_layers=num_layers)
        self.final = SeqLinear(hidden_size,output_size)
        
    def forward(self, x):
        out,z = self.rnn(x)
#         print(x.shape,z.shape,out.shape)
        out = self.final(out)
        return out
    

In [ ]:
model = SimpleIndRNN(2,1,6,hidden_size=100)
lrn = Learner(db,model,loss_func=nn.MSELoss())#,opt_func=ranger)
# lrn.add_cb(WeightClipping(model.rnn.list_rnn))
lrn.add_cb(IndRNNRegularizer())
# lrn.add_cb(GradientClipping())
lrn.fit(1,lr=2e-4)

NameError: name 'SeqLinear' is not defined

In [ ]:
model = SimpleIndRNN(2,1,6,hidden_size=100,bn=True,seq_len=1000)
lrn = Learner(db,model,loss_func=nn.MSELoss())#,opt_func=ranger)
# lrn.add_cb(WeightClipping(model.rnn.list_rnn))
lrn.add_cb(IndRNNRegularizer())
# lrn.add_cb(GradientClipping())
lrn.fit(1,lr=2e-4)

lrn.validate

In [ ]:
#hide
from nbdev.export import *
notebook2script()